In [1]:
import pandas as pd

In [2]:
class MultiCollinearityEliminator():

    # Class Constructor
    def __init__(self, df, target, threshold):
        self.df = df
        self.target = target
        self.threshold = threshold

    # Method to create and return the feature correlation matrix dataframe
    def createCorrMatrix(self, include_target=False):
        # Checking we should include the target in the correlation matrix
        if (include_target == False):
            df_temp = self.df.drop([self.target], axis=1)

            # Setting method to Pearson to prevent issues in case the default method for df.corr() gets changed
            # Setting min_period to 30 for the sample size to be statistically significant (normal) according to
            # central limit theorem
            corrMatrix = df_temp.corr(method='pearson', min_periods=30).abs()
        # Target is included for creating the series of feature to target correlation - Please refer the notes under the
        # print statement to understand why we create the series of feature to target correlation
        elif (include_target == True):
            corrMatrix = self.df.corr(method='pearson', min_periods=30).abs()
        return corrMatrix

    # Method to create and return the feature to target correlation matrix dataframe
    def createCorrMatrixWithTarget(self):
        # After obtaining the list of correlated features, this method will help to view which variables
        # (in the list of correlated features) are least correlated with the target
        # This way, out the list of correlated features, we can ensure to elimate the feature that is
        # least correlated with the target
        # This not only helps to sustain the predictive power of the model but also helps in reducing model complexity

        # Obtaining the correlation matrix of the dataframe (along with the target)
        corrMatrix = self.createCorrMatrix(include_target=True)
        # Creating the required dataframe, then dropping the target row
        # and sorting by the value of correlation with target (in asceding order)
        corrWithTarget = pd.DataFrame(corrMatrix.loc[:, self.target]).drop([self.target], axis=0).sort_values(
            by=self.target)
        #print(corrWithTarget, '\n')
        return corrWithTarget

    # Method to create and return the list of correlated features
    def createCorrelatedFeaturesList(self):
        # Obtaining the correlation matrix of the dataframe (without the target)
        corrMatrix = self.createCorrMatrix(include_target=False)
        colCorr = []
        # Iterating through the columns of the correlation matrix dataframe
        for column in corrMatrix.columns:
            # Iterating through the values (row wise) of the correlation matrix dataframe
            for idx, row in corrMatrix.iterrows():
                if (row[column] > self.threshold) and (row[column] < 1):
                    # Adding the features that are not already in the list of correlated features
                    if (idx not in colCorr):
                        colCorr.append(idx)
                    if (column not in colCorr):
                        colCorr.append(column)
        #print(colCorr, '\n')
        return colCorr

    # Method to eliminate the least important features from the list of correlated features
    def deleteFeatures(self, colCorr):
        # Obtaining the feature to target correlation matrix dataframe
        corrWithTarget = self.createCorrMatrixWithTarget()
        for idx, row in corrWithTarget.iterrows():
            #print(idx, '\n')
            if (idx in colCorr):
                self.df = self.df.drop(idx, axis=1)
                break
        return self.df

    # Method to run automatically eliminate multicollinearity
    def autoEliminateMulticollinearity(self):
        # Obtaining the list of correlated features
        colCorr = self.createCorrelatedFeaturesList()
        while colCorr != []:
            # Obtaining the dataframe after deleting the feature (from the list of correlated features)
            # that is least correlated with the taregt
            self.df = self.deleteFeatures(colCorr)
            # Obtaining the list of correlated features
            colCorr = self.createCorrelatedFeaturesList()
        return self.df

In [5]:
features = pd.read_csv('./features/AbAg_features.csv')
features = features.drop(['ID'], axis=1)

my_eliminator = MultiCollinearityEliminator(features, 'ddG', 0.97)
features = my_eliminator.autoEliminateMulticollinearity()

In [8]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
features

,Hyd_Hyd_4,Hyd_Pos_4,Hyd_Neg_4,Hyd_Acc_4,Hyd_Don_4,Hyd_Sul_4,Hyd_Neu_4,Pos_Hyd_4,Pos_Pos_4,Pos_Neg_4,Pos_Acc_4,Pos_Don_4,Pos_Aro_4,Pos_Sul_4,Pos_Neu_4,Neg_Hyd_4,Neg_Pos_4,Neg_Neg_4,Neg_Acc_4,Neg_Don_4,Neg_Aro_4,Neg_Sul_4,Neg_Neu_4,Acc_Hyd_4,Acc_Pos_4,Acc_Neg_4,Acc_Acc_4,Acc_Don_4,Acc_Aro_4,Acc_Sul_4,Acc_Neu_4,Don_Hyd_4,Don_Pos_4,Don_Neg_4,Don_Acc_4,Don_Don_4,Don_Aro_4,Don_Sul_4,Don_Neu_4,Aro_Pos_4,Aro_Neg_4,Aro_Acc_4,Aro_Don_4,Aro_Aro_4,Aro_Sul_4,Aro_Neu_4,Sul_Hyd_4,Sul_Pos_4,Sul_Neg_4,Sul_Acc_4,Sul_Don_4,Sul_Aro_4,Sul_Sul_4,Sul_Neu_4,Neu_Hyd_4,Neu_Pos_4,Neu_Neg_4,Neu_Acc_4,Neu_Don_4,Neu_Aro_4,Neu_Sul_4,Neu_Neu_4,Hyd_Hyd_8,Hyd_Pos_8,Hyd_Neg_8,Hyd_Aro_8,Hyd_Sul_8,Hyd_Neu_8,Pos_Hyd_8,Pos_Pos_8,Pos_Neg_8,Pos_Acc_8,Pos_Don_8,Pos_Aro_8,Pos_Sul_8,Neg_Hyd_8,Neg_Pos_8,Neg_Neg_8,Neg_Acc_8,Neg_Don_8,Neg_Aro_8,Neg_Sul_8,Acc_Pos_8,Acc_Neg_8,Acc_Acc_8,Acc_Don_8,Acc_Aro_8,Acc_Sul_8,Acc_Neu_8,Don_Pos_8,Don_Neg_8,Don_Acc_8,Don_Don_8,Don_Neu_8,Aro_Hyd_8,Aro_Pos_8,Aro_Neg_8,Aro_Acc_8,Aro_Aro_8,Aro_Sul_8,Sul_Hyd_8,Sul_Pos_8,Sul_Neg_8,Sul_Acc_8,Sul_Aro_8,Sul_Sul_8,Neu_Hyd_8,Neu_Acc_8,Neu_Don_8,Neu_Neu_8,Hyd_Hyd_12,Hyd_Pos_12,Hyd_Neg_12,Hyd_Acc_12,Hyd_Aro_12,Hyd_Sul_12,Pos_Hyd_12,Pos_Pos_12,Pos_Neg_12,Pos_Acc_12,Pos_Aro_12,Pos_Sul_12,Neg_Hyd_12,Neg_Pos_12,Neg_Neg_12,Neg_Acc_12,Neg_Don_12,Neg_Aro_12,Neg_Sul_12,Acc_Hyd_12,Acc_Pos_12,Acc_Neg_12,Acc_Acc_12,Acc_Don_12,Acc_Aro_12,Acc_Neu_12,Don_Neg_12,Don_Acc_12,Don_Sul_12,Don_Neu_12,Aro_Hyd_12,Aro_Pos_12,Aro_Neg_12,Aro_Acc_12,Aro_Aro_12,Aro_Sul_12,Sul_Hyd_12,Sul_Pos_12,Sul_Neg_12,Sul_Don_12,Sul_Aro_12,Sul_Sul_12,Neu_Acc_12,Neu_Don_12,Neu_Neu_12,Hyd_Hyd_16,Hyd_Pos_16,Hyd_Neg_16,Hyd_Acc_16,Hyd_Don_16,Hyd_Aro_16,Hyd_Sul_16,Hyd_Neu_16,Pos_Hyd_16,Pos_Pos_16,Pos_Neg_16,Pos_Acc_16,Pos_Don_16,Pos_Aro_16,Pos_Sul_16,Pos_Neu_16,Neg_Hyd_16,Neg_Pos_16,Neg_Neg_16,Neg_Acc_16,Neg_Don_16,Neg_Aro_16,Neg_Sul_16,Acc_Hyd_16,Acc_Pos_16,Acc_Neg_16,Acc_Acc_16,Acc_Don_16,Acc_Aro_16,Acc_Sul_16,Acc_Neu_16,Don_Hyd_16,Don_Pos_16,Don_Neg_16,Don_Acc_16,Don_Don_16,Don_Sul_16,Don_Neu_16,Aro_Hyd_16,Aro_Pos_16,Aro_Neg_16,Aro_Acc_16,Aro_Sul_16,Sul_Hyd_16,Sul_Pos_16,Sul_Neg_16,Sul_Acc_16,Sul_Don_16,Sul_Aro_16,Sul_Sul_16,Neu_Hyd_16,Neu_Pos_16,Neu_Acc_16,Neu_Don_16,Neu_Neu_16,Hyd_Hyd_20,Hyd_Pos_20,Hyd_Neg_20,Hyd_Acc_20,Hyd_Don_20,Hyd_Aro_20,Hyd_Sul_20,Hyd_Neu_20,Pos_Hyd_20,Pos_Pos_20,Pos_Neg_20,Pos_Acc_20,Pos_Don_20,Pos_Aro_20,Pos_Sul_20,Pos_Neu_20,Neg_Hyd_20,Neg_Pos_20,Neg_Neg_20,Neg_Acc_20,Neg_Don_20,Neg_Aro_20,Neg_Sul_20,Neg_Neu_20,Acc_Hyd_20,Acc_Pos_20,Acc_Neg_20,Acc_Acc_20,Acc_Don_20,Acc_Aro_20,Acc_Sul_20,Acc_Neu_20,Don_Hyd_20,Don_Pos_20,Don_Neg_20,Don_Acc_20,Don_Don_20,Don_Aro_20,Don_Sul_20,Don_Neu_20,Aro_Hyd_20,Aro_Pos_20,Aro_Neg_20,Aro_Acc_20,Aro_Don_20,Aro_Aro_20,Aro_Sul_20,Aro_Neu_20,Sul_Hyd_20,Sul_Pos_20,Sul_Neg_20,Sul_Acc_20,Sul_Don_20,Sul_Aro_20,Sul_Sul_20,Sul_Neu_20,Neu_Hyd_20,Neu_Pos_20,Neu_Neg_20,Neu_Acc_20,Neu_Don_20,Neu_Aro_20,Neu_Sul_20,Neu_Neu_20,∆Hyd,∆Pos,∆Neg,∆Acc,∆Don,∆Aro,∆Sul,∆Neu,∆AAhydropathy,∆AAarea,∆AAcharge,∆AAflexibily,∆AAchemical,∆AAsize,∆AAhbonds,ddG
0,91.0,6,6,45,52,1,64,6,0.0,0,1,0,0,0,2,6,0,3.0,8,4,0,0,9,45,1,8,10.0,37,10,1,71,52,0,4,37,11.0,12,0,64,0,0,10,12,31.0,0,0,1,0,0,1,0,0,0.0,0,64,2,9,71,64,0,0,52.0,406.0,21,49,264,16,435,21,0.0,7,18,13,8,0,49,7,4.0,21,21,17,0,18,21,54.0,105,54,3,166,13,21,105,50.0,188,264,8,17,54,52.0,6,16,0,0,3,6,0.0,435,166,188,154.0,426.0,54,112,392,196,21,54,1.0,4,14,18,0,112,4,5.0,42,36,38,2,392,14,42,79.0,140,128,224,36,140,10,218,196,18,38,128,8.0,8,21,0,2,10,8,0.0,224,218,175.0,257.0,16,111,297,227,133,9,424,16,0.0,1,8,8,2,2,16,111,1,3.0,41,41,28,0,297,8,41,58.0,100,95,3,174,227,8,41,100,38.0,5,126,133,2,28,95,0,9,2,0,3,5,0,0.0,424,16,174,126,96.0,45.0,3,22,55,38,8,3,74,3,0.0,0,1,2,0,0,5,22,0,0.0,8,12,1,4,26,55,1,8,9.0,14,7,5,37,38,2,12,14,2.0,3,1,12,8,0,1,7,3,0.0,0,7,3,0,4,5,1,0,0.0,3,74,5,26,37,12,7,3,19.0,0,0,-2,-2,0,0,0,-1,5.3,-35.0,1.0,-17.0,-2.0,-1.0,-3.0,0.48
1,89.0,9,16,64,57,0,94,9,1.0,3,4,3,0,0,8,16,3,3.0,9,6,0,0,12,64,4,9,20.0,58,7,0,109,57,3,6,58,18.0,6,0,87,0

In [10]:
features.to_csv('./features/AbAg_features_reduced.csv', index=False)